# **Reto I**

### 1. Datasets

Los datos de origen constan de dos archivos csv con la misma estructura y tipo de columnas.

* trade_details: dataset original con datos reales de operaciones financieras.
* trade_details_snapshot: copia de seguridad por posibles perdidas de datos.

### 2. Columnas y significado:

* mfamily: indica la familia de operaciones a la que pertenece.
* mgroup: indica el grupo de operaciones dentro de la familia.
* mtype: indica el tipo de operación dentro del grupo.
* origin_trade_number: indica el número de la operación de trading (la misma operación puede tener varios números de trading).
* origin_contract_number: indica el número de contrato de la operación (igual para todas las operaciones que pertenecen al mismo contrato).
* maturity: fecha de finalización del contrato de cada operación.

### 3. Descripción del problema:

En estos datasets se encuentran varias operaciones financieras de distinto tipo, que diferenciaremos mediante los distintos valores de las columnas mfamily, mgroup y mtype.

Existe un cierto tipo de operaciones especiales, llamadas FXSwaps. Estas pueden ser diferenciadas por medio de los siguientes valores:

**mfamily = CURR** \
**mgroup = FXD** \
**mtype = SWLEG**

Podemos ver en nuestro dataset que estas operaciones aparecen duplicadas, es decir, con el mismo **origin_contract_number** aunque distinto **origin_trade_number**. De estas operaciones duplicadas en origen, queremos obtener solo una de ellas.

La forma para decidir cuál de las operaciones nos interesa obtener es mediante la columna *maturity*. De ambas operaciones de trading (distinto origin_trade_number) para un mismo contrato (origin_contract_number), queremos obtener solo la *long leg*, es decir, la que tiene una mayor fecha de vencimiento (fecha más actual de la columna maturity).

Existe un cierto problema en nuestro dataset trade_details que tendremos que solucionar. Podemos ver que para algunas operaciones el campo maturity vendrá como *null*, es decir, sin informar. En estos casos, deberemos buscar esa operacion en el dataset trade_details_snapshot y el respectivo campo maturity para poder saber cuál de las dos operaciones es la *long leg* y filtrar la *short leg* 

**NOTA: Si se quiere conocer más el significado de estas operaciones financieras: https://es.wikipedia.org/wiki/Swap_(finanzas)**

### 4. Reto:

* Obtener un dataframe final donde tengamos todas las operaciones originales excepto los short leg de los contratos tipo FXSwap.
* Aunque usemos el valor de la columna maturity del dataset trade_details_snapshot en los casos que venga en la trade_details a *null*, en el dataframe final deberá venir con el valor original de trade_details.
* Hacerlo de la manera más eficiente posible a nivel computacional.

### Inicialización de SparkSession:

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Reto 1").master("local").getOrCreate()

### Carga de CSV

In [0]:
trade_details = spark.read\
                     .option("header", "true")\
                     .option("delimiter", ";")\
                     .csv("/FileStore/tables/trade_details.csv")

trade_details_snapshot = spark.read\
                              .option("header", "true")\
                              .option("delimiter", ";")\
                              .csv("/FileStore/tables/trade_details_snapshot.csv")

### Resultado:

**INSTRUCCIONES**: El DataFrame resultante debe almacenarse en la variable `resultado`, sustituyendo el valor `None` por el código que consideréis oportuno. De esta forma podréis comprobar si el resultado es correcto.

In [0]:
#hacemos un leftjoin
joinedDF = trade_details.join(trade_details_snapshot, trade_details.origin_trade_number == trade_details_snapshot.origin_trade_number, "left")\
                        .withColumn("maturity_completa", when(trade_details.maturity == "NULL", trade_details_snapshot.maturity).otherwise(trade_details.maturity))\
                        .select(trade_details.mfamily, trade_details.mgroup, trade_details.origin_trade_number,trade_details.mtype,trade_details.origin_contract_number, trade_details.maturity, "maturity_completa")
joinedDF.show()

+-------+------+-------------------+-----+----------------------+----------+-----------------+
|mfamily|mgroup|origin_trade_number|mtype|origin_contract_number|  maturity|maturity_completa|
+-------+------+-------------------+-----+----------------------+----------+-----------------+
|    IRD|  BOND|          316391872| null|             678876251|2021-09-22|       2021-09-22|
|   CURR|   FXD|           32734782|  FXD|              54853428|2021-09-22|       2021-09-22|
|    IRD| LN_BR|               1111| null|                  2222|2022-10-06|       2022-10-06|
|    IRD|   IRS|            2222222| null|               2222222|2024-10-15|       2024-10-15|
|    SCF|   SCF|            3815982|  SCF|               3672136|      NULL|       2021-05-04|
|    IRD| LN_BR|           14596583| null|              13774383|2020-12-29|       2020-12-29|
|    IRD|   IRS|           18343978| null|              17356077|2024-10-15|       2024-10-15|
|    IRD| LN_BR|           19203839| null|        

In [0]:
#dataframe solo con los fxswaps para luego editar los duplicados
fxswaps = joinedDF.filter((joinedDF["mfamily"]=="CURR") & (joinedDF["mgroup"] == "FXD") & (joinedDF["mtype"] == "SWLEG"))
fxswaps.show()

+-------+------+-------------------+-----+----------------------+----------+-----------------+
|mfamily|mgroup|origin_trade_number|mtype|origin_contract_number|  maturity|maturity_completa|
+-------+------+-------------------+-----+----------------------+----------+-----------------+
|   CURR|   FXD|           19665185|SWLEG|              18622136|2020-04-29|       2020-04-29|
|   CURR|   FXD|           19665186|SWLEG|              18622136|2020-12-30|       2020-12-30|
|   CURR|   FXD|           19772399|SWLEG|              18724280|      NULL|       2020-11-05|
|   CURR|   FXD|           19772400|SWLEG|              18724280|2021-11-05|       2021-11-05|
|   CURR|   FXD|           20980932|SWLEG|              19883451|      NULL|       2020-02-02|
|   CURR|   FXD|           20980933|SWLEG|              19883451|      NULL|       2020-01-19|
|   CURR|   FXD|           22798004|SWLEG|              21622649|      NULL|       2021-05-04|
|   CURR|   FXD|           22798005|SWLEG|        

In [0]:
#dataframe del resto para luego de editar el de fxswaps juntarlos nuevamente
resto = joinedDF.filter((joinedDF["mfamily"]!="CURR") | (joinedDF["mgroup"] != "FXD") | (joinedDF["mtype"] != "SWLEG"))
resto.show()

+-------+------+-------------------+-----+----------------------+----------+-----------------+
|mfamily|mgroup|origin_trade_number|mtype|origin_contract_number|  maturity|maturity_completa|
+-------+------+-------------------+-----+----------------------+----------+-----------------+
|    IRD|  BOND|          316391872| null|             678876251|2021-09-22|       2021-09-22|
|   CURR|   FXD|           32734782|  FXD|              54853428|2021-09-22|       2021-09-22|
|    IRD| LN_BR|               1111| null|                  2222|2022-10-06|       2022-10-06|
|    IRD|   IRS|            2222222| null|               2222222|2024-10-15|       2024-10-15|
|    SCF|   SCF|            3815982|  SCF|               3672136|      NULL|       2021-05-04|
|    IRD| LN_BR|           14596583| null|              13774383|2020-12-29|       2020-12-29|
|    IRD|   IRS|           18343978| null|              17356077|2024-10-15|       2024-10-15|
|    IRD| LN_BR|           19203839| null|        

In [0]:
#creamos columna de cantidad de fecha de hoy y cantidad de dias para poder ver cual es la fecha mas actual
import pyspark.sql.functions as F
from pyspark.sql.functions import *
fxswaps1 = fxswaps.withColumn("fecha_hoy", current_date())\
                  .withColumn("cantidad_dias", F.datediff("fecha_hoy", "maturity_completa"))
fxswaps1.show()

+-------+------+-------------------+-----+----------------------+----------+-----------------+----------+-------------+
|mfamily|mgroup|origin_trade_number|mtype|origin_contract_number|  maturity|maturity_completa| fecha_hoy|cantidad_dias|
+-------+------+-------------------+-----+----------------------+----------+-----------------+----------+-------------+
|   CURR|   FXD|           19665185|SWLEG|              18622136|2020-04-29|       2020-04-29|2022-05-05|          736|
|   CURR|   FXD|           19665186|SWLEG|              18622136|2020-12-30|       2020-12-30|2022-05-05|          491|
|   CURR|   FXD|           19772399|SWLEG|              18724280|      NULL|       2020-11-05|2022-05-05|          546|
|   CURR|   FXD|           19772400|SWLEG|              18724280|2021-11-05|       2021-11-05|2022-05-05|          181|
|   CURR|   FXD|           20980932|SWLEG|              19883451|      NULL|       2020-02-02|2022-05-05|          823|
|   CURR|   FXD|           20980933|SWLE

In [0]:
#hacemos un groupby para que nos quede solo los origin contract number con la fecha mas actual
fxswap3 = fxswaps1.groupBy("origin_contract_number").agg(F.min("cantidad_dias").alias("cantidad_dias"))
fxswap3.show()

+----------------------+-------------+
|origin_contract_number|cantidad_dias|
+----------------------+-------------+
|              19622128|          491|
|              21622649|          359|
|              19883451|          823|
|              18622136|          491|
|              18724280|          181|
+----------------------+-------------+



In [0]:
#juntamos la tabla del group by con la de fxswap1
resultado1 = fxswaps1.join(fxswap3, on= (fxswaps1["origin_contract_number"] == fxswap3["origin_contract_number"]) & (fxswaps1["cantidad_dias"] == fxswap3["cantidad_dias"]))\
                     .drop(fxswaps1["origin_contract_number"]).drop(fxswaps1["cantidad_dias"])
resultado1= resultado1.drop(*("fecha_hoy", "cantidad_dias")).select("mfamily","mgroup","origin_trade_number","mtype","origin_contract_number","maturity", "maturity_completa")
resultado1.show()


+-------+------+-------------------+-----+----------------------+----------+-----------------+
|mfamily|mgroup|origin_trade_number|mtype|origin_contract_number|  maturity|maturity_completa|
+-------+------+-------------------+-----+----------------------+----------+-----------------+
|   CURR|   FXD|           19665186|SWLEG|              18622136|2020-12-30|       2020-12-30|
|   CURR|   FXD|           19772400|SWLEG|              18724280|2021-11-05|       2021-11-05|
|   CURR|   FXD|           20980932|SWLEG|              19883451|      NULL|       2020-02-02|
|   CURR|   FXD|           22798005|SWLEG|              21622649|      NULL|       2021-05-11|
|   CURR|   FXD|           20665178|SWLEG|              19622128|2020-12-30|       2020-12-30|
+-------+------+-------------------+-----+----------------------+----------+-----------------+



In [0]:
#se realiza una union del join anterior con la tabla resto q habiamos separado al principio
resultado = resultado1.union(resto).select("mfamily", "mgroup", "origin_trade_number", "mtype", "origin_contract_number", "maturity")
resultado.show()

+-------+------+-------------------+-----+----------------------+----------+
|mfamily|mgroup|origin_trade_number|mtype|origin_contract_number|  maturity|
+-------+------+-------------------+-----+----------------------+----------+
|   CURR|   FXD|           19665186|SWLEG|              18622136|2020-12-30|
|   CURR|   FXD|           19772400|SWLEG|              18724280|2021-11-05|
|   CURR|   FXD|           20980932|SWLEG|              19883451|      NULL|
|   CURR|   FXD|           22798005|SWLEG|              21622649|      NULL|
|   CURR|   FXD|           20665178|SWLEG|              19622128|2020-12-30|
|    IRD|  BOND|          316391872| null|             678876251|2021-09-22|
|   CURR|   FXD|           32734782|  FXD|              54853428|2021-09-22|
|    IRD| LN_BR|               1111| null|                  2222|2022-10-06|
|    IRD|   IRS|            2222222| null|               2222222|2024-10-15|
|    SCF|   SCF|            3815982|  SCF|               3672136|      NULL|

In [0]:
resultado.filter((resultado["mfamily"]=="CURR") & (resultado["mgroup"] == "FXD") & (resultado["mtype"] == "SWLEG")).show()

+-------+------+-------------------+-----+----------------------+----------+
|mfamily|mgroup|origin_trade_number|mtype|origin_contract_number|  maturity|
+-------+------+-------------------+-----+----------------------+----------+
|   CURR|   FXD|           19665186|SWLEG|              18622136|2020-12-30|
|   CURR|   FXD|           19772400|SWLEG|              18724280|2021-11-05|
|   CURR|   FXD|           20980932|SWLEG|              19883451|      NULL|
|   CURR|   FXD|           22798005|SWLEG|              21622649|      NULL|
|   CURR|   FXD|           20665178|SWLEG|              19622128|2020-12-30|
+-------+------+-------------------+-----+----------------------+----------+



Ejecuta la siguiente celda (no modifiques su código) y te dirá si tu solución es correcta o no. En caso de ser correcta, se ejecutará correctamente y no mostrará nada, pero si no lo es mostrará un error. Además de esas pruebas, se realizarán algunas más (ocultas) a la hora de puntuar el ejercicio, pero evaluar dicha celda es un indicador bastante fiable acerca de si realmente has implementado la solución correcta o no.

In [0]:
assert(resultado.count() == 26)
assert(resultado.orderBy("origin_contract_number").collect()[24][4] == "564367838")
assert(resultado.orderBy("origin_contract_number").collect()[19][5] == "NULL")
assert(resultado.orderBy("origin_trade_number").collect()[16][5] == "NULL")